In [3]:
import networkx as nx
from linkages import *
import numpy as np
import openmesh as om
import scipy

def nullspace(A, eps=1e-12):
    u, s, vh = np.linalg.svd(A)
    padding = max(0,np.shape(A)[1]-np.shape(s)[0])
    null_mask = np.concatenate(((s <= eps), np.ones((padding,),dtype=bool)),axis=0)
    null_space = np.compress(null_mask, vh, axis=0)
    return np.transpose(null_space)

def nullspace_not_working(A, eps=1e-15):
    u, s, vh = np.linalg.svd(A)
    null_mask = (s <= eps)
    null_space = np.compress(null_mask, vh, axis=0)
    return np.transpose(null_space)

def nullspace_doof(A, rcond=None):
    u, s, vh = np.linalg.svd(A, full_matrices=True)
    M, N = u.shape[0], vh.shape[1]
    if rcond is None:
        rcond = np.finfo(s.dtype).eps * max(M, N)
    tol = np.amax(s) * rcond
    num = np.sum(s > tol, dtype=int)
    Q = vh[num:,:].T.conj()
    return Q

def graph_to_matrix(G):
    rows = []
    for edge in G.edges:
        row = []
        d = edge[0] - edge[1]
        for vertex in G.nodes:
            if vertex == edge[0]  : row.extend(d)
            elif vertex == edge[1]: row.extend(-d)
            else:                   row.extend([0 for _ in range(DIM)])
        rows.append(row)
    return np.matrix(rows, float)

def set_pinning(pins, M: np.matrix):
    N = nullspace(M)
    if type(pins) is int: pins = [pins]

    for p in pins:
        for vector in N.A:
            if vector.size == 0:
                continue
            for i in range(DIM):
                if vector[DIM*(p-1) + i] != 0:
                    vector.fill(0)
                    break
    return N

# helper function to convert nullspace to a list of motions 
def getMotions(N):
    if type(N) is np.matrix: N = nullspace(N)
    return [[str(val) + "*v" + str(np.floor(i/DIM)+1) + ("xyzwrüdiger"[i%DIM]) for i,val in enumerate(vector) if np.abs(val) > 1e-15] for vector in N.A]

#just a cute function to convert detected motions into a human readible string
def motions_to_string(motions):
    string = ""
    for v in motions:
        if len(v) !=0:
            for val in v:
                if val == v[0] and len(v)>1: word = " depends on "
                elif val != v[len(v)-1]    : word = ", and "
                elif len(v) == 1           : word = " is free\n"
                else                       : word ="\n"
                string += val + word
    return string

def check_rigidity(M): return np.linalg.matrix_rank(M) == M.shape[1]- (DIM+1) * DIM/2

def model_to_graph(mesh: om.PolyMesh) -> nx.Graph:
    graph = nx.Graph()

    points = mesh.points()
    for edge in mesh.edge_vertex_indices():
        graph.add_edge(Point(points[edge[0]]), Point(points[edge[1]]))

    return graph    


graph_cool = model_to_graph(om.read_polymesh("models/cube.stl"))


A = graph_to_matrix(graph_cool)
print(A.shape)
print ("the linkage is infinitesimally rigid!" if check_rigidity(A) else "the linkage is infinitesimally flexible")
A = set_pinning(1,A)

print(motions_to_string((getMotions(A))))



(18, 24)
the linkage is infinitesimally rigid!



![Matrix](matrix)

![Matrix](matrix)